In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import itertools
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

まずは導入  
Macの環境では「esc + h」でショートカットを確認可能  
「esc + m」でマークダウンに変更可能  
「esc + y」でコードに変更可能

In [6]:
train = pd.read_csv('../input/ch01-titanic/train.csv')
test = pd.read_csv('../input/ch01-titanic/test.csv')

学習データ、テストデータの読み込みを行なっている。

In [7]:
train_x = train.drop(['Survived'], axis = 1)
train_y = train['Survived']

学習データは特徴量と目的変数に分けている。

In [8]:
test_x = test.copy()

テストデータは特徴量のみなのでそのままコピーしている。
pythonのコピーについて「deepcopy」とかいうのがあるらしくcopyとの違いがわからないので参考urlを貼っておく。
https://murashun.jp/article/programming/python/python-list-copy-deepcopy.html

# 特徴量作成

In [9]:
train_x = train_x.drop(['PassengerId'], axis = 1)
test_x = test_x.drop(['PassengerId'], axis = 1)

変数PassengerIdを除外している。

In [10]:
train_x = train_x.drop(['Name', 'Ticket', 'Cabin'], axis = 1)
test_x = test_x.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

変数Name, Ticket, Cabinを除外する。

In [11]:
for c in ['Sex', 'Embarked']:
    # 学習データに基づいてどう変換するかを定める
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))
    
    # 学習データ、テストデータを変換する
    train_x[c] = le.transform(train_x[c].fillna('NA'))
    test_x[c] = le.transform(test_x[c].fillna('NA'))

それぞれのカテゴリ変数にlabel encodeingを適用している。

Label Encoder:　データ（特にラベル、質的データについて数値に変換する）
le.fit(data):学習
le.transform(data):適応

説明url:https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

# モデル作成

In [12]:
model = XGBClassifier(n_estimators = 20, random_state = 71)
model.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=20, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=71,
              reg_alpha=0, reg_lambda=1, ...)

モデルの生成および学習データの予測値を確率で出力する。

In [13]:
pred = model.predict_proba(test_x)[:, 1]

テストデータ予測値を確率で出力する。

In [15]:
pred_label = np.where(pred > 0.5, 1, 0)

テストデータの予測値を二値に変換する。

In [18]:
submission = pd.DataFrame(
    {'PassengerId': test['PassengerId'], 'Survived': pred_label})
submission.to_csv('submission_first.csv', index = False)

スコア:0.7799(本書中の数値と異なる可能性がある)

# バリデーション

In [21]:
scores_accuracy = []
scores_logloss = []

各foldのスコアを保存するリスト

In [24]:
kf = KFold(n_splits = 4, shuffle = True, randam_state = 71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], tarin_y.iloc[va_idx]
    
    #モデルの学習を行う
    model = XGBClassifier(n_estimators = 20, random_state = 71)
    model.fit(tr_x, tr_y)
    
    #バリデーションデータの予測値を確率で出力する
    va_pred = model.predict_prob(va_x)[:, 1]
    
    #バリデーションデータでのスコアを計算する
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)
    
    #そのfoldのスコアを保存する
    scores_logloss.a

TypeError: __init__() got an unexpected keyword argument 'randam_state'